# Building a Data Lakehouse with AWS Lake Formation and Apache Iceberg

In this lab, you will create a Data Lakehouse with a medallion architecture using LakeFormation and Apache Iceberg tables. Your source system will be a 
relational database instantiated as MySQL database in Amazon RDS (Relational Database Service) and a streaming service that stores data in Amazon S3. You will first populate the medallion architecture with AWS Glue jobs, using Apache Iceberg tables for certain tables. You will finally query the data stored in the latest layer using Amazon Athena. To define and configure the components of this data pipeline example, you will use Terraform as Infrastructure as Code (IaC) service.

# Table of Contents

- [ 1 - Introduction and Setup](#1)
  - [ 1.1 - Introduction](#1.1)
  - [ 1.2 - Setting up the Data Lakehouse](#1.2)
- [ 2 - Architecture of the Data Lakehouse](#2)
- [ 3 - Landing Zone ](#3)
  - [ 3.1 - RDS and Streaming Landing](#3.1)
  - [ 3.2 - Deployment](#3.2)
- [ 4 - Curated Zone](#4)
  - [ 4.1 - CSV Transformation](#4.1)
  - [ 4.2 - JSON Transformation and Apache Iceberg](#4.2)
  - [ 4.3 - Deployment](#4.3)
- [ 5 - Presentation zone](#5)
- [ 6 - [Optional and Not Graded] - Some Features of Iceberg Format](#6)
  - [ 6.1 - Schema Evolution](#6.1)
  - [ 6.2 - Versioning with Iceberg](#6.2)
- [ 7 - [Optional and Not Graded] - Granting Permissions with Lake Formation](#7)
- [ 8 - Enviroment Clean Up](#8)

First, let's import some libraries that you need for the lab.

In [1]:
import json

import boto3
import pandas as pd
import awswrangler as wr
from scripts import lf_utils

<a name='1'></a>
## 1 - Introduction and Setup

<a name='1.1'></a>
### 1.1 - Introduction

Assume you work as a Data Engineer at a retailer of scale models of classic cars and other transportation media. The retailer stores its historical purchases and its 
customers' information in a relational database that consists of the following tables: customers, products, productlines, orders, orderdetails, payments, employees, offices. You will use the same database example as in some previous labs: [MySQL Sample Database](https://www.mysqltutorial.org/mysql-sample-database.aspx)

The new web page of the company had a streaming service setup to quickly retrieve the ratings that users give to different products, the company has decided to migrate data architecture to the cloud. The relational database was recreated in RDS and the setup of a streaming sink to output the data was done in S3 bucket. You need to set up a Data Lakehouse, using medallion architecture to organize the data in three different zones:

- **Landing**: Zone where raw data arrives from the RDS and the streaming service.
- **Curated**: The raw data is enriched, curated and added to the Glue Data Catalog.
- **Presentation**: Business objects are built on top of the curated data.

<a name='1.2'></a>
### 1.2 - Setting up the Data Lakehouse

The Data Lake will be based on the S3 bucket, the name of the bucket is one of the outputs in CloudFormation (_Data Lake Bucket_). Your first job is to create folders to separate the bucket by the three zones:

- `landing_zone`
- `curated_zone`
- `presentation_zone`

Next, you will be using LakeFormation to manage the Data Lakehouse, most of the setup has been done to assign the admins of the Data Lakehouse. You will have to grant permission to the Glue Job role (`Cloud9-de-c3w2lab2-glue-role`) and the role to the instance where you are running Cloud9 (`Cloud9InstanceRole`) to be able to work inside the Data Lakehouse. A simple way to set up the permissions is using `boto3`. Start from the setup for the permissions using the first role.

1.2.1. Set up some boto3 clients.

In [2]:
lf_client = boto3.client('lakeformation', region_name='us-east-1')
iam_client = boto3.client('iam', region_name='us-east-1')
glue_client = boto3.client("glue", region_name='us-east-1')

INFO:botocore.credentials:Found credentials from IAM Role: LabInstanceRole


1.2.2. Create necessary constant variables. Go to the AWS console, and search for **CloudFormation**; you will see two stacks, one that has an alphanumeric ID and another one with a name starting with `aws-cloud9`. Click on the stack with the alphanumeric ID and then click on the **Outputs** tab. In the list of outputs, you will see the bucket with the CloudFormation *Key* `DataLakeBucket`. Copy the corresponding bucket name, that you can find under the column *Value*. In the code below, replace the placeholder `<DATA_LAKE_BUCKET>` with the copied value and run the code:

In [19]:
BUCKET_NAME = 'de-c3w2a1-381492175033-us-east-1-data-lake'
DATA_LAKE_BUCKET = BUCKET_NAME
CURATED_DATABASE_NAME = 'curated_zone'
PRESENTATION_DATABASE_NAME = 'presentation_zone'
DATA_LOCATION_ARN = f"arn:aws:s3:::{BUCKET_NAME}"

1.2.3. You need the ARN of the roles or users whom you are granting permissions. The required functions are in the Python file `scripts/lf_utils.py`.

You will use the `iam_client` and the `lf_utils.get_role_arn` function. You first will grant data location access on the _Data Lake Bucket_ passing the ARN from the bucket to the `lf_utils.grant_data_location_access`, then you will grant access to the databases using `lf_utils.grant_database_access`. The two databases have already been set up in the AWS Glue Catalog.

In [5]:
glue_job_arn = lf_utils.get_role_arn(iam_client,'Cloud9-de-c3w2a1-glue-role')
lf_utils.grant_data_location_access(lf_client, glue_job_arn, DATA_LOCATION_ARN)
lf_utils.grant_database_access(lf_client, glue_job_arn, CURATED_DATABASE_NAME)
lf_utils.grant_database_access(lf_client, glue_job_arn, PRESENTATION_DATABASE_NAME)

INFO:root:Grant Data location access to: arn:aws:iam::381492175033:role/Cloud9-de-c3w2a1-glue-role 

INFO:root:Grant database curated_zone access to:arn:aws:iam::381492175033:role/Cloud9-de-c3w2a1-glue-role

INFO:root:Grant database presentation_zone access to:arn:aws:iam::381492175033:role/Cloud9-de-c3w2a1-glue-role



1.2.4. Grant the same permissions on the data lakehouse resources to the instance role.

In [6]:
instance_role_arn = lf_utils.get_role_arn(iam_client,'LabInstanceRole')
lf_utils.grant_data_location_access(lf_client, instance_role_arn, DATA_LOCATION_ARN) 
lf_utils.grant_database_access(lf_client, instance_role_arn, CURATED_DATABASE_NAME)
lf_utils.grant_database_access(lf_client, instance_role_arn, PRESENTATION_DATABASE_NAME)

INFO:root:Grant Data location access to: arn:aws:iam::381492175033:role/LabInstanceRole 

INFO:root:Grant database curated_zone access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant database presentation_zone access to:arn:aws:iam::381492175033:role/LabInstanceRole



<a name='2'></a>
## 2 - Architecture of the Data Lakehouse

Here's the proposed architecture of the Data Lakehouse. This architecture can serve as a solution to the problem introduced in this lab. It extracts the data from both source systems and uses AWS Glue jobs to transform and store the data into the different medallion zones, making it queryable for the data analyst through Amazon Athena.

![lake_diagram](images/Datalakehouse.png)

Here is a brief description of the components:

- **Data Sources:** You have already interacted with the source database in previous labs, the streaming service brings `ratings` data into an S3 bucket which you will call `source_bucket`. 

- **Medallion Layers:** As discussed previously, the architecture will be separated into three layers. The Glue jobs will be in charge of loading, transforming and storing the data from the source systems already located in AWS until the curated_zone.

  - **Landing layer:** This step involves extracting data from the OLTP database in RDS and from the streaming service's S3 bucket and storing the raw data in the `landing_zone`.
    - RDS Landing: An AWS Glue Job is used to connect to the RDS database and retrieve the data in CSV format. 
    - Streaming Landing: An AWS Glue Job is used to ingest the data from the `source_bucket` in S3, which is stored in JSON format. 

  - **Curated layer:** After extracting the data, Glue jobs will perform data transformation on top of the raw data. In addition to the transformation, the new curated sources will be stored in the `curated_zone` and added to the Glue Catalog.
    -  CSV Transformation: For the RDS tables, they will be enriched with metadata, the schema will be enforced and then the data will be stored in parquet format. 
    -  JSON Transformation: While the streaming data, as the JSON data containing ratings,  will be joined with the data from the OLTP system to create a table to be used for the ML team. This data will be stored as an Apache Iceberg table. 

  - **Presentation layer:** [Amazon Athena](https://aws.amazon.com/en/athena/), a serverless, interactive analytics service, is used to query the data stored in the `curated_zone` using the Glue Catalog. It enables SQL-like queries without needing to extract the data from S3 and load it into a traditional database. You will create presentation tables in this layer to solve some of the reporting and analytical needs using SQL queries, storing the data in Apache Iceberg format in the `presentation_zone`.

- **End Users:** You will also be in charge of managing the Data Lakehouse in the role of administration, you will have to grant access to the data to one user: an employee from the Machine Learning team (`ML_User`). 

<a name='3'></a>
## 3 - Landing Zone 

To start the Data Lakehouse you need to bring the raw data into the Data Lake. You will use Glue Jobs to do it. The scripts for these jobs are located in `terraform/assets/landing_etl_jobs`. You will use Terraform to deploy them.

<a name='3.1'></a>
### 3.1 - RDS and Streaming Landing

3.1.1. Open the script `terraform/assets/landing_etl_jobs/de_c3w2a1_batch_ingress.py`. The code is complete, try to understand it using the comments. 

3.1.2. To deploy the corresponding AWS Glue Job with Terraform, create the Glue Connection to the RDS database. Open the file `terraform/modules/landing_etl/glue.tf` and search for `resource "aws_glue_connection" "rds_connection"`. In the `connection_properties`, complete the [map object](https://developer.hashicorp.com/terraform/language/expressions/types) with the following key-value pairs:

- Set `JDBC_CONNECTION_URL` to  `"jdbc:mysql://${var.host}:${var.port}/${var.database}"`
- Set `USERNAME` to `var.username`
- Set `PASSWORD` to `var.password`

Make sure that you save the file after making changes in all of the steps for this lab.

3.1.3. Open the script `terraform/assets/landing_etl_jobs/de_c3w2a1_json_ingress.py`. The code is complete, go through the lines of the code with the comments to understand it generally.

<a name='3.2'></a>
### 3.2 - Deployment

3.2.1. Open the `terraform/main.tf` file and make sure that only the `landing_zone` module is uncommented (lines 1-16). 

3.2.2. Go to the `terraform` folder in the Cloud9 or Jupyter terminal, initialize Terraform and deploy the resources running the following commands:

```bash
cd terraform
terraform init
terraform plan
terraform apply
```

*Note*: You will need to type in `yes` and press `Enter` to perform actions.

You will get the outputs from Terraform, including two job names: `glue_bucket_ingestion_job` and `glue_rds_ingestion_job`.

3.2.3. Use the Glue Job names (`de-c3w2a1-bucket-ingestion-etl-job` and `de-c3w2a1-rds-ingestion-etl-job`) from the Terraform output to run each job:

```bash
aws glue start-job-run --job-name <GLUE-JOB-NAME> | jq -r '.JobRunId'
```

You should get `JobRunID` in the output. Check the status of the AWS Glue Jobs exchanging the placeholder `<JOB-RUN-ID>` with the output from the previous step:

```bash
aws glue get-job-run --job-name <GLUE-JOB-NAME> --run-id <JOB-RUN-ID> --output text --query "JobRun.JobRunState"
```

In [7]:
!aws glue start-job-run --job-name de-c3w2a1-bucket-ingestion-etl-job | jq -r '.JobRunId'

jr_671656de03147e90016cde74d153718352f196394bb81f417d03a5971b9e9adb


In [10]:
!aws glue start-job-run --job-name de-c3w2a1-rds-ingestion-etl-job | jq -r '.JobRunId'

jr_7e4a6c523888d810a4f3a7babbdb7464fe1eeb1e8380442a393628d0363420ae


In [16]:
!aws glue get-job-run --job-name de-c3w2a1-bucket-ingestion-etl-job --run-id jr_671656de03147e90016cde74d153718352f196394bb81f417d03a5971b9e9adb --output text --query "JobRun.JobRunState"

SUCCEEDED


In [18]:
!aws glue get-job-run --job-name de-c3w2a1-rds-ingestion-etl-job --run-id jr_7e4a6c523888d810a4f3a7babbdb7464fe1eeb1e8380442a393628d0363420ae --output text --query "JobRun.JobRunState"

SUCCEEDED


When each run job has a `SUCCEEDED` status, you can continue with the rest of the lab. Each of them should take around 2 minutes to complete. You can run them at the same time.

3.2.4. You can inspect the result of your Glue Jobs by running the `aws cli` command, either in the following cells or in the terminal. Make sure to replace the placeholder <DATA_LAKE_BUCKET> with the actual name of your S3 _Data Lake Bucket_ before running the command.

In [21]:
!aws s3 ls s3://{DATA_LAKE_BUCKET}/landing_zone/rds/

                           PRE customers/
                           PRE employees/
                           PRE offices/
                           PRE orderdetails/
                           PRE orders/
                           PRE payments/
                           PRE productlines/
                           PRE products/


In [22]:
!aws s3 ls s3://{DATA_LAKE_BUCKET}/landing_zone/json/

                           PRE ratings/


<a name='4'></a>
## 4 - Curated Zone

Once you have checked that the data is available in the `landing_zone`. You can continue with the transformations that will be performed over your data to be stored at the `curated_zone`. 

<a name='4.1'></a>
### 4.1 - CSV Transformation

4.1.1. Go to the `terraform/assets/transform_etl_jobs` and there you will find the three scripts with which you will work to feed the data into the second layer of your Data Lake. The script `de_c3w2a1_batch_transform.py` will be used to perform the transformation job: add metadata and enforce the schema, stored in CSV format. The file already has the majority of the logic you need, you just need to complete a couple of functions.

4.1.2. **Add Metadata**. Metadata refers to data that describes data, it can be related to lineage, source, history or version. Complete the function `add_metadata` to add the required metadata following the instructions in the code.

4.1.3. **Enforce schema**. Schema enforcement is essential in data engineering as it ensures data consistency and integrity by enforcing predefined data types, structures and constraints. It helps to prevent errors and facilitates seamless data integration and analysis processes. In this case, the Glue Job contains the schema for each table as a `StructType` object. For the columns that are not of type `StringType`, you want to verify that the corresponding column in the`source_pd` Dataframe has the same type. Complete the function `enforce_schema` following the instructions in the code.

<a name='4.2'></a>
### 4.2 - JSON Transformation and Apache Iceberg

For the transformation of the data that arrives in JSON format, you will execute two Glue Jobs:

- One to join the JSON data with the CSV files that you stored in the landing zone.
- A second job will take the JSON data and store it in another file format.

For the two jobs, the result will be stored in the Apache Iceberg format. The Apache Iceberg format is an open standard designed for handling large analytical datasets efficiently. It's designed to provide fast, consistent, and reliable access to large datasets in cloud storage. Under the hood, Iceberg leverages a combination of file formats, metadata management, and transactional capabilities to provide efficient access to large datasets. It typically stores data using columnar file formats such as Parquet or ORC. These formats are optimized for analytical queries. On the other hand, Iceberg maintains extensive metadata alongside the actual data files. This metadata includes information about the table schema, partitioning strategy, file locations, and transaction state. By storing metadata separately from the data files, Iceberg ensures efficient management and access to table metadata.

Some of the main features of the Apache Iceberg format are:
- **Schema Flexibility**: Iceberg facilitates schema evolution, enabling the addition or removal of table columns without necessitating a full dataset rewrite. This ensures seamless backward compatibility as data structures evolve over time.
- **Transactional Integrity**: Iceberg tables ensure atomic commits, guaranteeing that all table changes are transactional. This ensures that updates are either fully executed or entirely rolled back, maintaining data consistency and reliability.
- **Data Partitioning**: Iceberg supports data partitioning based on one or more columns, enhancing query performance by minimizing data scan volumes. Particularly beneficial for large datasets.
- **Comprehensive Metadata Management**: Iceberg maintains comprehensive metadata alongside the data, including schema details, partitioning specifications, and file locations. This metadata is stored separately from the data files, simplifying management and queries.

Let's prepare the scripts for the Glue Job.

4.2.1. First, you will save the data that is in JSON format into Apache Iceberg. Open the file `terraform/assets/transform_etl_jobs/de_c3w2a1_ratings_to_iceberg.py` and search for the variable named `SqlQuery0`. Complete the provided query with the following:

```sql
select * 
from ratings 
where ingest_ts = (select max(ingest_ts) from ratings)
;
```

It will pull out the ratings with the most recent `ingest_ts` timestamp.

4.2.2. Have a look at the query stored in the variable `SqlQuery1`. You will have to modify it in a later stage in the lab, but in the meantime, try to understand the purpose of it.

4.2.3. Open the file located at `terraform/modules/transform_etl/glue.tf`. Search for the `resource "aws_glue_job" "ratings_to_iceberg_job"` resource. Complete the following parameters:

- Set the `timeout` parameter to 7.
- Set the number of workers to 2.
- In `default_arguments`, set the `--job-language` argument to `"python"`. Create another parameter named `--datalake-formats` and set the value to `"iceberg"` in the last line of the default parameters.

4.2.4. Open the glue script `terraform/assets/transform_etl_jobs/de_c3w2a1_json_transform.py`. This script is in charge of unifying some data that you extracted from the RDS Database with the ratings in JSON format so the ML Team can use it to create some models. Search for the variable named `SqlQuery1` and complete the query following the instructions in the code.

<a name='4.3'></a>
### 4.3 - Deployment

4.3.1. Open the file `terraform/main.tf` and uncomment the module named `transform_etl` (lines 18-32).

4.3.2. Open the file `terraform/outputs.tf` and uncomment the outputs associated with the `transform_etl` module (lines 15-25).

4.3.3. Deploy the Glue Jobs following the same steps as in section [3.2](#3.2). You will need to execute three Glue Jobs: `glue_csv_transform_job`, `glue_ratings_transform_job`, `glue_ratings_to_iceberg_job` (job names `de-c3w2a1-csv-transformation-job`, `de-c3w2a1-ratings-transformation-job` and `de-c3w2a1-ratings-to-iceberg-job`). Each of them should take around 3 minutes, you can run them at the same time.

In [29]:
!aws glue start-job-run --job-name de-c3w2a1-csv-transformation-job | jq -r '.JobRunId'

jr_216c93ef4c505077e16a4c240d3f62e2522ddaf448c11e2b6fa3c0af5f3abc41


In [30]:
!aws glue start-job-run --job-name de-c3w2a1-ratings-transformation-job | jq -r '.JobRunId'

jr_c7d1fb018c7941a301643bf2705f36b25d198669930555c5af4ee9b1fdee98f3


In [31]:
!aws glue start-job-run --job-name de-c3w2a1-ratings-to-iceberg-job | jq -r '.JobRunId'

jr_ad0f1c6c11dd7936918f6c90af40327279707c47db69e91041996b88c6229134


4.3.4. When each run job has a `SUCCEEDED` status, you can continue with the rest of the lab. Use the `aws s3 ls` command to inspect again the result of your transformations.

In [34]:
!aws s3 ls s3://{DATA_LAKE_BUCKET}/curated_zone/ratings/iceberg

                           PRE iceberg/


In [35]:
!aws s3 ls s3://{DATA_LAKE_BUCKET}/curated_zone/customers

                           PRE customers/


<a name='5'></a>
## 5 - Presentation zone

For the presentation zone, you will use AWS Athena. You will be using the `awswrangler` library to run `CREATE TABLE AS` queries on the `presentation_zone`, this is an example of how it works. 

*Note*: `awswrangler` uses the `pyarrow` library that has some functions returning a `FutureWarning` due to deprecation, this doesn't affect what you are trying to do so you can filter out those warnings.

In [36]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

5.1. First, grant access to the tables in the `curated_zone` to the instance role. You will use a Glue client with `boto3` to obtain the table names in the catalog.

In [37]:
curated_tables = glue_client.get_tables(DatabaseName=CURATED_DATABASE_NAME)
curated_tables_list = curated_tables["TableList"]
curated_table_names = [tableDict["Name"] for tableDict in curated_tables_list]
for table in curated_table_names:
    lf_utils.grant_table_access(lf_client, instance_role_arn, CURATED_DATABASE_NAME, table)

INFO:root:Grant table customers access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table employees access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table offices access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table orderdetails access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table orders access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table payments access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table productlines access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table products access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table ratings access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table ratings_for_ml access to:arn:aws:iam::381492175033:role/LabInstanceRole



5.2. This is an example of creating an Iceberg Table based on a query using Athena and the `curated_zone` tables.

In [38]:
ctas_query = f"""CREATE TABLE ratings WITH (
	table_type = 'ICEBERG',
	location = 's3://{BUCKET_NAME}/presentation_zone/ratings',
	is_external = false
) AS
SELECT *
FROM {CURATED_DATABASE_NAME}.ratings;"""
response = wr.athena.start_query_execution(
    sql=ctas_query,
    database=PRESENTATION_DATABASE_NAME,
    wait=True,
    s3_output = f's3://{BUCKET_NAME}/athena_output/ratings'
)
print(response['Status']['State'])

SUCCEEDED


5.3. In the previous example, you made the `ratings` table available in the `presentation` layer for data analysts to use. Bring the table `ratings_for_ml` from the `curated zone` into the `presentation` layer, cast the `process_ts` column to `varchar`.

In [39]:
ctas_query = f"""CREATE TABLE ratings_for_ml WITH (
	table_type = 'ICEBERG',
	location = 's3://{BUCKET_NAME}/presentation_zone/ratings_for_ml',
	is_external = false
) AS
SELECT customerNumber, city, state, postalCode, country, creditLimit, productCode,productLine, productScale, quantityinstock, buyprice, msrp, productRating, cast(process_ts as varchar) as process_ts
FROM {CURATED_DATABASE_NAME}.ratings_for_ml;"""
response = wr.athena.start_query_execution(
    sql=ctas_query,
    database=PRESENTATION_DATABASE_NAME,
    wait=True,
    s3_output = f's3://{BUCKET_NAME}/athena_output/ratings_for_ml'
)
print(response['Status']['State'])

SUCCEEDED


5.4. Now you will create tables that may involve running aggregations or joins across multiple tables from the curated zone. The first one will involve returning the average sales per month and year and storing them in the table `sales_report`.

In [40]:
ctas_query = f"""CREATE TABLE sales_report WITH (
	table_type = 'ICEBERG',
	location = 's3://{BUCKET_NAME}/presentation_zone/sales_report',
	is_external = false
) AS select year(orderdate) as sales_year, month(orderdate) as sales_month, sum(od.quantityordered * od.priceeach) as sales_total
from {CURATED_DATABASE_NAME}.orders o
left join {CURATED_DATABASE_NAME}.orderdetails od
on o.ordernumber = od.ordernumber
group by year(orderdate), month(orderdate)
order by year(orderdate), month(orderdate);"""
response = wr.athena.start_query_execution(
    sql=ctas_query,
    database=PRESENTATION_DATABASE_NAME,
    wait=True,
    s3_output = f's3://{BUCKET_NAME}/athena_output/sales_report'
)
print(response['Status']['State'])

SUCCEEDED


5.5. For the second business object, create a table called `ratings_per_product`, where you calculate the average rating and review count per each product. The table must return the product code, product name, average rating and count of reviews. Order the table by review count in descending order and then by average count.

In [41]:
ctas_query = f"""
CREATE TABLE ratings_per_product WITH (
	table_type = 'ICEBERG',
	location = 's3://{BUCKET_NAME}/presentation_zone/ratings_per_product',
	is_external = false
) AS SELECT 	
p.productcode,
p.productname,
avg(productrating) as avg_rating,
count(*) as review_count
FROM {CURATED_DATABASE_NAME}.products p
LEFT JOIN {CURATED_DATABASE_NAME}.ratings r
ON p.productcode = r.productcode
GROUP BY p.productcode,p.productname
ORDER BY review_count DESC,avg_rating;
"""
response = wr.athena.start_query_execution(
    sql=ctas_query,
    database=PRESENTATION_DATABASE_NAME,
    wait=True,
    s3_output = f's3://{BUCKET_NAME}/athena_output/ratings_per_product'
)
print(response['Status']['State'])

SUCCEEDED


5.6. Let's check on the new tables. Grant permissions to the instance role on all the tables in the `presentation_zone` database and then query each table, bringing 10 records for each.

In [42]:
presentation_tables = glue_client.get_tables(DatabaseName=PRESENTATION_DATABASE_NAME)
presentation_tables_list = presentation_tables["TableList"]
presentation_table_names = [tableDict["Name"] for tableDict in presentation_tables_list]
for table in presentation_table_names:
    lf_utils.grant_table_access(lf_client, instance_role_arn, PRESENTATION_DATABASE_NAME, table)

INFO:root:Grant table avg_ratings_per_location access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table ratings access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table ratings_for_ml access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table ratings_per_product access to:arn:aws:iam::381492175033:role/LabInstanceRole

INFO:root:Grant table sales_report access to:arn:aws:iam::381492175033:role/LabInstanceRole



In [43]:
sql = 'SELECT * FROM ratings LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    s3_output = f's3://{BUCKET_NAME}/athena_output/sample_ratings'
)
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "presentation_zone"."temp_table_30d1b2f92806458786df907dd8998f74"


,customernumber,productcode,productrating,ingest_ts
0,103,S10_2016,3,2024-11-16T18:37:42.463Z
1,103,S24_2022,5,2024-11-16T18:37:42.463Z
2,112,S24_1046,4,2024-11-16T18:37:42.463Z
3,112,S24_3969,4,2024-11-16T18:37:42.463Z
4,112,S18_1889,4,2024-11-16T18:37:42.463Z


In [44]:
sql = 'SELECT * FROM ratings_for_ml LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    s3_output = f's3://{BUCKET_NAME}/athena_output/sample_ratings_for_ml'
)
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "presentation_zone"."temp_table_3f127941131d4cae9ec40dafa76834d6"


,customernumber,city,state,postalcode,country,creditlimit,productcode,productline,productscale,quantityinstock,buyprice,msrp,productrating,process_ts
0,314,Bruxelles,<NA>,B-1180,Belgium,79900,S18_3320,Vintage Cars,1:18,7913,58,99,3,2024-11-16 18:59:33.418
1,114,Melbourne,Victoria,3004,Australia,117300,S12_1666,Trucks and Buses,1:12,1579,78,137,5,2024-11-16 18:59:33.418
2,276,North Sydney,NSW,2060,Australia,107800,S12_1666,Trucks and Buses,1:12,1579,78,137,4,2024-11-16 18:50:10.957
3,276,North Sydney,NSW,2060,Australia,107800,S18_1589,Classic Cars,1:18,9042,66,124,3,2024-11-16 18:50:10.957
4,276,North Sydney,NSW,2060,Australia,107800,S18_2319,Trucks and Buses,1:18,8258,75,123,3,2024-11-16 18:50:10.957


In [45]:
sql = 'SELECT * FROM sales_report LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    s3_output = f's3://{BUCKET_NAME}/athena_output/sample_sales_report'
)
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "presentation_zone"."temp_table_72aebbf527af4e0ca1cb93efdbf12949"


,sales_year,sales_month,sales_total
0,2003,1,116689.21875
1,2003,4,185835.21875
2,2003,5,179435.90625
3,2003,6,150462.71875
4,2003,7,201947.46875


In [46]:
sql = 'SELECT * FROM ratings_per_product LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    s3_output = f's3://{BUCKET_NAME}/athena_output/sample_ratings_per_product'
)
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "presentation_zone"."temp_table_3afb6f49fdc54f3cae5e8315f43c74e4"


,productcode,productname,avg_rating,review_count
0,S18_3232,1992 Ferrari 360 Spider red,3.115385,26
1,S18_1129,1993 Mazda RX-7,2.777778,18
2,S24_1578,1997 BMW R 1100 S,2.588235,17
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,2.823529,17
4,S18_1342,1937 Lincoln Berline,2.882353,17


<a name='6'></a>
## 6 - [Optional and Not Graded] - Some Features of Iceberg Format

<a name='6.1'></a>
### 6.1 - Schema Evolution

Now, you will see some of the features that the Iceberg format has. You will focus mostly on the schema evolution and querying data from different versions. The team that serves the data in the S3 source bucket has told you that they added a new column holding the date at which the customers performed the rating to the product, named `ratingtimestamp`. The data is hosted in the same bucket, but at the S3 key named `ratings_with_timestamp`. This will be the folder where all new data will continue to be stored. Your task is to add the new column to your `ratings` table at the `curated_zone` database and then extract the new data. Let's use the `awswrangler` package again to perform some queries over AWS Athena. 

6.1.1. Explore the current available tables in your `curated_zone` database:

In [47]:
wr.catalog.tables(database=CURATED_DATABASE_NAME)

,Database,Table,Description,TableType,Columns,Partitions
0,curated_zone,customers,,EXTERNAL_TABLE,"customernumber, customername, contactlastname,...",
1,curated_zone,employees,,EXTERNAL_TABLE,"employeenumber, lastname, firstname, extension...",
2,curated_zone,offices,,EXTERNAL_TABLE,"officecode, city, phone, addressline1, address...",
3,curated_zone,orderdetails,,EXTERNAL_TABLE,"ordernumber, productcode, quantityordered, pri...",
4,curated_zone,orders,,EXTERNAL_TABLE,"ordernumber, orderdate, requireddate, shippedd...",
5,curated_zone,payments,,EXTERNAL_TABLE,"customernumber, checknumber, paymentdate, amou...",
6,curated_zone,productlines,,EXTERNAL_TABLE,"productline, textdescription, htmldescription,...",
7,curated_zone,products,,EXTERNAL_TABLE,"productcode, productname, productline, product...",
8,curated_zone,ratings,,EXTERNAL_TABLE,"customernumber, productcode, productrating, in...",
9,curated_zone,ratings_for_ml,,EXTERNAL_TABLE,"customernumber, city, state, postalcode, count...",


6.1.2. You can see the `ratings` table there with some of the current columns. Query the table `ratings` at its current state. Create a query to select all the columns from the `ratings` table; limit your search to 10 rows. Save this query as a string into the `sql` variable and then execute the query.

In [48]:
sql = "SELECT * FROM ratings LIMIT 10;"
df = wr.athena.read_sql_query(sql, database=CURATED_DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "curated_zone"."temp_table_5d92693024ca46ef8110f601fcd12eae"


,customernumber,productcode,productrating,ingest_ts
0,103,S10_2016,3,2024-11-16T18:37:42.463Z
1,103,S24_2022,5,2024-11-16T18:37:42.463Z
2,112,S24_1046,4,2024-11-16T18:37:42.463Z
3,112,S24_3969,4,2024-11-16T18:37:42.463Z
4,112,S18_1889,4,2024-11-16T18:37:42.463Z


6.1.3. Now, let's add the new column to the `iceberg` table. Open the file `terraform/assets/alter_table_job/de_c3w2a1_alter_ratings_table.py` and search for the function named `add_column()`. There you will see an SQL Statement that performs an `ALTER TABLE` to add a new column. The column name and type are passed as parameters to the Glue Job. 

6.1.4. Open the file `terraform/main.tf` and uncomment the module named `alter_table` (lines 34-49).

6.1.5. Open the file `terraform/outputs.tf` and uncomment the outputs associated with the `alter_table` module (lines 28-30).

6.1.6. Deploy the Glue Job following the same steps as in section 3.2. The Glue Job name will be in the new output `glue_alter_table_job`. It will take about 2 minutes to run.

In [49]:
!aws glue start-job-run --job-name de-c3w2a1-alter-table-job | jq -r '.JobRunId'

jr_672712ef68f5e1c888233e0488b56abc1eb97a9ec805b6e11e44c1a8c22c800f


6.1.7. Once your job has finished with a `SUCCEEDED` status, run the following cell to check that the new column has been added.

In [50]:
sql = "SELECT * FROM ratings LIMIT 10;"
df = wr.athena.read_sql_query(sql, database=CURATED_DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "curated_zone"."temp_table_a2edf110abdb4db99cdaf34eb04d427c"


,customernumber,productcode,productrating,ingest_ts,ratingtimestamp
0,103,S10_2016,3,2024-11-16T18:37:42.463Z,<NA>
1,103,S24_2022,5,2024-11-16T18:37:42.463Z,<NA>
2,112,S24_1046,4,2024-11-16T18:37:42.463Z,<NA>
3,112,S18_1342,5,2024-11-16T18:37:42.463Z,<NA>
4,114,S10_2016,4,2024-11-16T18:37:42.463Z,<NA>


6.1.8. The table schema has been changed, you will need to modify your previous Glue Job to take into account such new changes. Open the Glue Job script `terraform/assets/landing_etl_jobs/de_c3w2a1_json_ingress.py`. Search for the creation of the object named `source_ratings_json`. You can see that in the `connection_options` there is a `"paths"` parameter. Change the key in the source data lake from `ratings` to `ratings_with_timestamp`, save the file. 

6.1.9. Execute again the Terraform scripts with:

```bash
terraform plan
terraform apply
```

Remember to write `yes` to apply the changes. In this case, although your infrastructure wouldn't experiment any change, this new deployment will upload the changes in your glue script. 

6.1.10. Run again the corresponding Glue Job associated with the terraform output `glue_bucket_ingestion_job`. Wait until the job has finished with the `SUCCEEDED` status (it will take about 2 minutes).

In [51]:
!aws glue start-job-run --job-name de-c3w2a1-bucket-ingestion-etl-job | jq -r '.JobRunId'

jr_4853bfacf3d03e85153254e75c1b900bbfe7ec18b7c9f62f150da70788dec51d


6.1.11. Now that the new ratings with the new column have arrived in your `landing_zone`, it is time to use them in your next jobs. For the sake of simplicity, you will only run the job that transforms the data from JSON format into `iceberg`. Open the `terraform/assets/transform_etl_jobs/de_c3w2a1_ratings_to_iceberg.py` file and search for the query that performs a merge into your database. In the `UPDATE SET` statement, add the `ratingtimestamp` column using the same syntax that you can see for the other two columns. 

6.1.12. Execute again the Terraform scripts and run the `glue_ratings_to_iceberg_job`. It will take about 3 minutes to run.

In [52]:
!aws glue start-job-run --job-name de-c3w2a1-ratings-to-iceberg-job | jq -r '.JobRunId'

jr_0c8c5df678ece032a440b8792b5e3cb47c2cc86a6ca3968be54088ae91185c83


6.1.13. Once the job has finished successfully, execute again a query over the `ratings` table to see the new results.

In [53]:
sql = "SELECT * FROM ratings WHERE ratingtimestamp IS NOT NULL LIMIT 10;"
df = wr.athena.read_sql_query(sql, database=CURATED_DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "curated_zone"."temp_table_262bdb340f5042a085c9b157feb4241e"


,customernumber,productcode,productrating,ingest_ts,ratingtimestamp
0,319,S18_3232,3,2024-11-16T19:11:57.538Z,2023-12-08
1,319,S10_4698,5,2024-11-16T19:11:57.538Z,2023-12-23
2,319,S700_2466,5,2024-11-16T19:11:57.538Z,2024-01-10
3,319,S700_3167,2,2024-11-16T19:11:57.538Z,2024-01-13
4,320,S18_1662,4,2024-11-16T19:11:57.538Z,2023-09-10


Now that you have realized that the schema of your table has evolved and you were able to add data with a new schema you will explore another feature of the `iceberg` format.

<a name='6.2'></a>
### 6.2 - Versioning with Iceberg

One of the advantages of the `iceberg` format is that it allows you to save different versions of your tables. Those versions are stored as snapshots. To check the number of versions/snapshots of your `ratings` table, which is the one that has changed, you can execute the following query:

In [54]:
sql = 'SELECT CAST(committed_at AS VARCHAR) as committed_at, snapshot_id,  parent_id, operation, manifest_list FROM "ratings$snapshots" order by committed_at asc;'
df = wr.athena.read_sql_query(sql, database=CURATED_DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "curated_zone"."temp_table_4e1aa822a2c04e3e853bb0bf5bf5f854"


,committed_at,snapshot_id,parent_id,operation,manifest_list
0,2024-11-16 18:50:35.361 UTC,1483002535559935149,<NA>,overwrite,s3://de-c3w2a1-381492175033-us-east-1-data-lak...
1,2024-11-16 18:59:54.282 UTC,8510594347790662512,1483002535559935149,overwrite,s3://de-c3w2a1-381492175033-us-east-1-data-lak...
2,2024-11-16 19:14:38.122 UTC,494767752037017630,8510594347790662512,overwrite,s3://de-c3w2a1-381492175033-us-east-1-data-lak...


You can see snapshot ID, the time at which that snapshot was created (`committed_at`) and also a `parent_id`, which corresponds to the previous version of the `snapshot_id`. To check the data that was stored at a particular version of your table, you can add the `FOR VERSION AS OF` statement at the end of your query, followed by the snapshot ID that you want to query. Complete the following query with the snapshot ID of your first table version (this is the one that has no `parent_id`) replacing the placeholder `<FIRST-SNAPSHOT-ID>`. Then complete another cell with the last snapshot ID (replacing `<LAST-SNAPSHOT-ID>`); to request the data of the last version of your data. You can execute the two of them to compare.

In [56]:
sql = 'SELECT * FROM ratings FOR VERSION AS OF 1483002535559935149;'
df = wr.athena.read_sql_query(sql, database=CURATED_DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "curated_zone"."temp_table_22c5d9b7ea774d27a2dd08987dfee20b"


,customernumber,productcode,productrating,ingest_ts
0,103,S10_2016,3,2024-11-16T18:37:42.463Z
1,103,S24_2022,5,2024-11-16T18:37:42.463Z
2,112,S24_1046,4,2024-11-16T18:37:42.463Z
3,119,S10_1678,3,2024-11-16T18:37:42.463Z
4,119,S18_2581,4,2024-11-16T18:37:42.463Z


In [57]:
sql = 'SELECT * FROM ratings FOR VERSION AS OF 494767752037017630;'
df = wr.athena.read_sql_query(sql, database=CURATED_DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "curated_zone"."temp_table_60037e5a6b624e5c87dec0891c823c7f"


,customernumber,productcode,productrating,ingest_ts,ratingtimestamp
0,319,S18_3232,3,2024-11-16T19:11:57.538Z,2023-12-08
1,103,S10_2016,3,2024-11-16T18:37:42.463Z,<NA>
2,319,S10_4698,5,2024-11-16T19:11:57.538Z,2023-12-23
3,319,S700_2466,5,2024-11-16T19:11:57.538Z,2024-01-10
4,319,S700_3167,2,2024-11-16T19:11:57.538Z,2024-01-13


With this exercise, you have used the data versioning feature of Apache Iceberg which allows users to track changes to datasets over time by creating immutable snapshots of the data at different points in time. Two important points to take into account are:

- When a user creates a new version of a dataset in Iceberg, Iceberg creates an immutable snapshot of the dataset's state at that point in time. This snapshot includes all the data files and metadata associated with the dataset.
- On the other hand, Iceberg maintains metadata to track the history of dataset versions. Each version is assigned a unique identifier, typically a timestamp or version number, and includes information about the changes made in that version.

*Note*: **The next sections is optional and not graded. If you choose to skip it, please submit the lab by clicking on the Submit button in the Vocareum lab window. Section 8 outlines the clean up process in case if you want to rerun some Terraform commands (also not graded).**

<a name='7'></a>
## 7 - [Optional and Not Graded] - Granting Permissions with Lake Formation

In your company there is a Machine Learning team that will use the results of your pipelines, in particular, they will use the `ratings_for_ml` table that is stored in the `presentation_zone` database. They use the AWS user `ml_data_lake_user` to interact with AWS resources and query the data from the `ratings_for_ml` table. Your task is to provide them only with the necessary access to the information, which means, they will only be able to query the `ratings_for_ml` table and shouldn't have access to any other table in the `presentation_zone` database. In the same way, they shouldn't be able to modify the data in that table.

You have access to the credentials of `ml_data_lake_user` to test the permissions that you will grant in this section. Run the following cell to grab the credentials from the AWS Secret Manager service.

In [58]:
secrets_client = boto3.client('secretsmanager')
response = secrets_client.get_secret_value(SecretId='/datalake/credentials/ml_data_lake_user')
crendentials = json.loads(response['SecretString'])

Now, you will create a `boto3` session with those credentials to access AWS resources as `ml_data_lake_user`.

In [59]:
session = boto3.Session(aws_access_key_id = crendentials['ACCESS_KEY'], aws_secret_access_key = crendentials['SECRET_KEY'])

Pass the `boto3` session you just created to the `read_sql_query` method below through the `boto3_session` parameter and execute the query. The `ml_data_lake_user` user doesn't have permission over the Data Lake Bucket so you will use an AWS Athena workgroup to execute the query. This query is expected to fail due to insufficient Lakeformation Permissions.

In [60]:
sql = 'SELECT * FROM ratings_for_ml LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    workgroup='de-c3w2a1-workgroup', 
    boto3_session=session)
df.head()

AccessDeniedException: An error occurred (AccessDeniedException) when calling the DeleteTable operation: Insufficient Lake Formation permission(s): Required Drop on temp_table_c012d2945f8344939ea49a86e5cf43a5

Now, you will provide access to the `presentation_zone` database and the `ratings_for_ml` table. For that, you require the `arn` of the `ml_data_lake_user`. Execute the following command with the corresponding user name to grab its arn. Save this value in the `ml_arn` variable in the subsequent cell replacing the placeholder `<ML_USER_ARN>` and execute it.

In [61]:
!aws iam get-user --user-name ml_data_lake_user

{
    "User": {
        "Path": "/",
        "UserName": "ml_data_lake_user",
        "UserId": "AIDAVRUVUUC46NLJAJMVW",
        "Arn": "arn:aws:iam::381492175033:user/ml_data_lake_user",
        "CreateDate": "2024-11-16T15:43:47+00:00",
        "Tags": [
            {
                "Key": "cloudlab",
                "Value": "c62825a2937394l8412135t1w381492175033"
            }
        ]
    }
}


In [62]:
ml_arn = 'arn:aws:iam::381492175033:user/ml_data_lake_user'
lf_utils.grant_database_access(lf_client, ml_arn, PRESENTATION_DATABASE_NAME)

INFO:root:Grant database presentation_zone access to:arn:aws:iam::381492175033:user/ml_data_lake_user



Finally, although the user has access to the `presentation_zone` database, it wouldn't be able to query the `ratings_for_ml` table directly. You have to provide direct access to that table using the `lf_utils.grant_table_access()` function. Remember that the `ml_data_lake_user` should be able to only read the data. Pass the table name and the list ['SELECT'] to the function below and execute it. After that, run the query again to check that the user has the appropriate access.

In [63]:
lf_utils.grant_table_access(lf_client, ml_arn, PRESENTATION_DATABASE_NAME, 'ratings_for_ml', ['SELECT'])

INFO:root:Grant table ratings_for_ml access to:arn:aws:iam::381492175033:user/ml_data_lake_user



In [64]:
session = boto3.Session(aws_access_key_id = crendentials['ACCESS_KEY'], aws_secret_access_key = crendentials['SECRET_KEY'])

sql = 'SELECT * FROM ratings_for_ml LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    workgroup='de-c3w2a1-workgroup', 
    boto3_session=session)
df.head()

INFO:awswrangler.athena._utils:Created CTAS table "presentation_zone"."temp_table_d0d0d2ab9be34425909fe50906e4dd94"


,customernumber,city,state,postalcode,country,creditlimit,productcode,productline,productscale,quantityinstock,buyprice,msrp,productrating,process_ts
0,314,Bruxelles,<NA>,B-1180,Belgium,79900,S18_3320,Vintage Cars,1:18,7913,58,99,3,2024-11-16 18:59:33.418
1,114,Melbourne,Victoria,3004,Australia,117300,S12_1666,Trucks and Buses,1:12,1579,78,137,5,2024-11-16 18:59:33.418
2,276,North Sydney,NSW,2060,Australia,107800,S12_1666,Trucks and Buses,1:12,1579,78,137,4,2024-11-16 18:50:10.957
3,276,North Sydney,NSW,2060,Australia,107800,S18_1589,Classic Cars,1:18,9042,66,124,3,2024-11-16 18:50:10.957
4,276,North Sydney,NSW,2060,Australia,107800,S18_2319,Trucks and Buses,1:18,8258,75,123,3,2024-11-16 18:50:10.957


As a final health check, you can run the following query with another table from the `presentation_zone` database to check that the user does not have read access to it. Change the placeholder `<PRESENTATION_LAYER_TABLE>` with the table name that you want to choose.

In [67]:
sql = 'SELECT * FROM ratings_per_product LIMIT 10;'
df = wr.athena.read_sql_query(
    sql, 
    database=PRESENTATION_DATABASE_NAME, 
    workgroup='de-c3w2a1-workgroup', 
    boto3_session=session)
df.head()

QueryFailed: TABLE_NOT_FOUND: line 7:3: Table 'awsdatacatalog.presentation_zone.ratings_per_product' does not exist. You may need to manually clean the data at location 's3://de-c3w2a1-381492175033-us-east-1-athena-wg/tables/12ada909-8786-4d8f-9118-0ee7fa8fdbdc' before retrying. Athena will not delete data in your account.

The medallion architecture offers several advantages for organizing and managing a data lakehouse effectively. This architecture enables the decoupling of storage and computing, allowing users to scale each layer independently based on their requirements. With this separation, organizations can optimize costs by utilizing cost-effective storage solutions while leveraging powerful computing resources for data processing and analytics tasks. In this particular case, you leveraged Amazon S3 as your data lake and storage solution and Amazon Glue for the computing required to transform the data, at the end you used Athena to create the business objects required by the organization to perform crucial analytics.

Lakeformation, a service provided by AWS, complements the medallion architecture by offering robust tools for securing and managing access to data within the data lake. Data governance is crucial for ensuring data integrity, compliance, and security, and Lakeformation plays a pivotal role by providing tools and capabilities to enforce governance policies effectively across a data lakehouse environment.

Please submit the lab by clicking on **Submit** button in the Vocareum lab window.

<a name='8'></a>
## 8 - Enviroment Clean Up

*Note*: **This section is required only if you want to rerun the Terraform commands. To receive a proper grade for this lab, please make sure that you have submitted the lab in the Vocareum lab window before performing the clean up steps outlined in this section.**

There might be cases when you will want to rerun Terraform commands. To avoid any issues with the already created resources, it is recommended to clean up environment before that. To start the cleanup, run the following command inside the `terraform` folder:

```bash
terraform destroy
```

As you have seen in this lab, AWS LakeFormation manages access to several data locations, at the end of the lab you need to restore the LakeFormation permissions to the default and revoke all the granted permissions. For this, you will call a Lambda function to perform the reset operation, use the following command in the Cloud9 or Jupyter terminal:

```bash
aws lambda invoke --function-name de-c3w2a1-Lambda-Setup --cli-binary-format raw-in-base64-out --payload '{ "RequestType": "Delete" }' --invocation-type Event response.json
```

You can check the status of your execution by using the following steps:

- First, get the log stream name with the following command:

```bash
aws logs describe-log-streams --log-group-name '/aws/lambda/de-c3w2a1-Lambda-Setup' --query logStreams[*].logStreamName
```

- This will return you the stream names associated with your execution. Copy the last one and paste it in the following command replacing the `<LOG-STREAM-NAME>` placeholder:

```bash
aws logs get-log-events --log-group-name '/aws/lambda/de-c3w2a1-Lambda-Setup' --log-stream-name '<LOG-STREAM-NAME>'
```

- You will be able to inspect the content of your log stream. The last part of the message should look similar to the following output:

```json
    {
        "timestamp": 1716912975732,
        "message": "END RequestId: 8dc59dd1-cbca-4c2a-912b-3bcfc5049962\n",
        "ingestionTime": 1716912984756
    },
    {
        "timestamp": 1716912975732,
        "message": "REPORT RequestId: 8dc59dd1-cbca-4c2a-912b-3bcfc5049962\tDuration: 22984.81 ms\tBilled Duration: 22985 ms\tMemory Size: 128 MB\tMax Memory Used: 92 MB\t\n",
        "ingestionTime": 1716912984756
    }
```

If you see the `END` of the request and the corresponding `REPORT`, that means that your lambda has finished. While you wait for your lambda execution to finish.